In [140]:
import requests
import pandas as pd
import numpy as np
import json

In [135]:
df = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv")

### Get the google api key
If you don't have one, create one on https://developers.google.com/maps/documentation/geocoding/get-api-key?hl=fr
and store it in a file called credentials.json

In [64]:
json_data=open('credentials.json').read()
data = json.loads(json_data)
API_KEY = data["google_api_key"]

In [146]:
url = "https://maps.googleapis.com/maps/api/geocode/json?address="
#address = "12 rue deu repos Lyon"
key = "&key=" +  API_KEY

def getLocation(address):
    response = requests.get(url+str(address)+key)
    if response.json()['status']=="OK":
        lat = response.json()['results'][0]['geometry']['location']['lat']
        lng = response.json()['results'][0]['geometry']['location']['lng']
    else:
        lat = np.nan
        lng = np.nan
    return pd.Series({'lat':lat,'lng':lng})

In [147]:
df = df.merge(df.Location.apply(lambda address: getLocation(address)), left_index=True, right_index=True)

In [149]:
df.to_csv('aircrashes1.csv')